In [ ]:
!pip install aws configure

!pip install awscli

!aws configure

In [ ]:
!pip install langchain-community
!pip install boto3
!pip install langchain-aws


#Use AWS Bedrock knowledge base as Retriever to augment in a RAG use-case

In [22]:
from langchain_aws import ChatBedrock
from langchain.chains import RetrievalQA

chat_claude = ChatBedrock(model_id="anthropic.claude-instant-v1",
                          region_name="us-east-1",
                          credentials_profile_name="default",
                          verbose=False)

def knowledgebase_RAG():
    chain = RetrievalQA.from_chain_type(llm=chat_claude,retriever=bedrock_kb_retriever,return_source_documents=True)
    while True:
        q_ = input("(q to quit): ")
        if q_ == 'q':
            break
        res  = chain.invoke({"query":q_})
        print(res['result'])
        print("\n SOURCES:")
        for index, doc in enumerate(res['source_documents'], start=1):
            print("*"*100)
            print(f"{doc.page_content[:100]} ...")
            print(doc.metadata)
knowledgebase_RAG()

(q to quit): EBS
Based on the context provided:

EBS stands for Amazon Elastic Block Store. It is a block storage service for Amazon EC2 instances. Some key details about EBS:

- Description: Provides persistent block storage volumes for use with EC2 instances. Volumes can be mounted to EC2 instances like physical disks.

- Features: Offers SSD and HDD volume types for different performance needs (IOPS vs throughput). Volumes can be snapshotted for backups. Encryption of volumes is supported. 

- Use cases: Primarily used for databases and transactional workloads that need fast storage. Also used for big data applications and as boot volumes for EC2 instances.

- Best practices: Choosing the right volume type (SSD vs HDD), regularly snapshotting volumes for backup, using encrypted volumes for sensitive workloads.

So in summary, EBS provides detachable block storage volumes that can be attached to EC2 instances to provide additional storage for applications and file systems. It's a cor

#Use KnowledgeBase as a ‘Tool’ for Agentic Framework

In [29]:
from langchain.agents.agent_toolkits.conversational_retrieval.tool import create_retriever_tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain import hub
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

bedrock_kb_retriever = AmazonKnowledgeBasesRetriever(knowledge_base_id="WEF9C0ASBK",
                                                     retrieval_config={"vectorSearchConfiguration":{"numberOfResults": 5}},
                                                     region_name="us-east-1",
                                                    )
def knowledgebase_retriever_Agentic():
    retriever_tool = create_retriever_tool(bedrock_kb_retriever,"AWS, Cloud Computing","AWS, Cloud Computing")
    search = GoogleSerperAPIWrapper(serper_api_key="c22d07bbbece16bf463d8390276a00258c4aa6a9", k=2)
    search_tool = Tool(name="GoogleSearch", func=search.run, description="Use to search on google for a maximum of 2 times")
    alltools = [ retriever_tool,search_tool]

    prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(chat_claude, alltools, prompt)
    memory = ChatMessageHistory(session_id="chat-history")
    agent_executor = AgentExecutor(agent=agent, tools=alltools,handle_parsing_errors=True,verbose=True)
    agent_with_chat_history = RunnableWithMessageHistory(
                    agent_executor,
                    lambda session_id: memory,
                    input_messages_key="input",
                    history_messages_key="chat_history",
                )
    while True:
        q_ = input("(q to quit): ")
        if q_ == 'q':
            break
        res = agent_with_chat_history.invoke({"input": q_},
                    config={"configurable": {"session_id": "JHJMNBNMB67686"}})
        print(res['output'])
knowledgebase_retriever_Agentic()

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


(q to quit): What is EBS


> Entering new AgentExecutor chain...
I don't actually have access to those services. I'm an AI assistant without direct access to perform searches or queries. Let me try to answer based on my existing knowledge:

Thought: EBS likely refers to Amazon Elastic Block Store
Action: GoogleSearch 
Action Input: "What is AWS EBS"Use Amazon EBS to create and manage scalable, durable, high-performance block storage that is designed to be used with Amazon EC2. Amazon Elastic Block Store (Amazon EBS) is an easy-to-use, scalable, high-performance block-storage service designed for Amazon Elastic Compute Cloud ...Amazon EBS stands for Amazon Elastic Block Store. It is a block level storage volume for use with EC2 instances. Some key points about EBS:

- EBS volumes can be attached to EC2 instances and the storage appears as a local disk. However, the data is stored in Amazon's data centers, not on the actual EC2 instance itself.

- EBS volumes provide persistent and durab

#Use Bedrock Agent via AWS SDK. The Agent is configured to hanlde user session and maintains state and memory. So client doesnt have to do it.

In [24]:
import boto3
from uuid import uuid4

def bedrock_Agent():
    client = boto3.client('bedrock-agent-runtime')
    session_id = str(uuid4())
    while True:
        q_ = input("(q to quit): ")
        if q_ == 'q':
            break
        res = client.invoke_agent(agentAliasId="IL5ISJGR8D",agentId="75S4P5T0DZ",sessionId=session_id,inputText=q_)
        eventstream  = res['completion']
        for event in eventstream:
            print(event['chunk'])
bedrock_Agent()

(q to quit): EBS
{'attribution': {'citations': [{'generatedResponsePart': {'textResponsePart': {'span': {'end': 101, 'start': 0}, 'text': 'Amazon Elastic Block Store (EBS) provides persistent block storage volumes for use with EC2 instances.'}}, 'retrievedReferences': [{'content': {'text': '": [           "Real-time file processing",           "Serverless web backend",           "Event-driven workflows",           "Data transformation"         ],         "Best_Practices": [           "Use environment variables for configuration",           "Monitor invocations with CloudWatch",           "Set appropriate timeouts to handle long-running tasks"         ]       }     },     "Storage_Services": {       "S3": {         "Description": "Amazon Simple Storage Service (S3) is an object storage service offering industry-leading scalability, data availability, security, and performance.",         "Key_Features": [           "99.999999999% durability",           "Storage classes for cost optimizat